In [91]:
from math import inf
from math import sqrt

import bisect
import gensim
import pandas as pd
import numpy as np
import pyemd
import nltk
from gensim.models import Word2Vec
from nltk import RegexpTokenizer as rpt
from nltk.corpus import stopwords as sw
from string import punctuation 

nltk.download('punkt')
nltk.download('stopwords')
stopwords = sw.words('portuguese')

model = Word2Vec.load('pt.bin')
model.init_sims(replace=True)

data_url="./results.csv"
data = pd.read_csv(data_url).replace(np.nan, '', regex=True)
data.head(5)

[nltk_data] Downloading package punkt to /home/vinha/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/vinha/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/home/vinha/workspace/w2v/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


,title,subtitle,author,date,section,text,url
0,“A sociedade foi Rubens Paiva não os facínora...,A decisão da juíza que proíbe as Forças Armada...,F. M.,30/03/2019 00:11:08,Brasil,A juíza federal Ivani Silva da Luz de Brasíli...,https://brasil.elpais.com/brasil/2019/03/26/po...
1,Justiça suspende decisão que proibia Forças Ar...,Liminar havia sido concedida na sexta-feira a ...,Marina Rossi,30/03/2019 16:17:59,Brasil,Menos de 24 horas depois de a juíza federal Iv...,https://brasil.elpais.com/brasil/2019/03/30/po...
2,Governo Bolsonaro prega “negacionismo históric...,Marcos Napolitano professor da USP diz que o...,Regiane Oliveira,04/04/2019 22:37:48,Brasil,Quando determinou que de 31 de março 1964 u...,https://brasil.elpais.com/brasil/2019/04/05/po...
3,Quando os pais de Gabo perceberam que tinham u...,Gustavo Tatis percorre o universo de García Má...,Jesús Ruiz Mantilla,07/03/2019 16:38:56,Cultura,Quando era pequeno Luisa e Gabriel se preo...,https://brasil.elpais.com/brasil/2019/03/06/cu...
4,Rádios canadenses banem músicas de Michael Jac...,Quebec Cogeco Media toma a decisão após queixa...,Jaime Porras Ferreyra,07/03/2019 16:12:37,Cultura,Desde a manhã da última segunda-feira e ...,https://brasil.elpais.com/brasil/2019/03/06/cu...


## Implemente uma função que recebe uma notícia e retorna os vetores (word embeddings) das palavras do título dessa notícia a partir dos word embeddings pré-treinados com o gensim (30 pts).

In [92]:
def parse(text):
    words = []
    word_pattern = rpt(r'\w+')
    year_pattern = rpt(r'\d{4}')
    
    patterns = [word_pattern, year_pattern]
    
    for pattern in patterns:
        tokens = []
        for token in pattern.tokenize(text):
            if token not in stopwords and len(token) > 3:
                tokens.append(token.lower())
        words.extend(tokens)
    return words

data['parsed_title'] = data['title']
def parse_title(x):
    x['parsed_title'] = parse(x.parsed_title)
    return x

def parse_vectors(x):
    aux = []
    for word in x.embeddings:
        try:
            aux.append(model[word])
        except KeyError:            
            aux.append([])
    x['embeddings'] = aux
    return x

data.apply(parse_title, axis=1)
data['embeddings'] = data['parsed_title']
data.apply(parse_vectors, axis=1)
data.head(5)

/home/vinha/workspace/w2v/lib/python3.6/site-packages/ipykernel_launcher.py:25: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


,title,subtitle,author,date,section,text,url,parsed_title,embeddings
0,“A sociedade foi Rubens Paiva não os facínora...,A decisão da juíza que proíbe as Forças Armada...,F. M.,30/03/2019 00:11:08,Brasil,A juíza federal Ivani Silva da Luz de Brasíli...,https://brasil.elpais.com/brasil/2019/03/26/po...,"[sociedade, rubens, paiva, facínoras, mataram]","[[0.0088292025, -0.1398486, -0.08332149, 0.084..."
1,Justiça suspende decisão que proibia Forças Ar...,Liminar havia sido concedida na sexta-feira a ...,Marina Rossi,30/03/2019 16:17:59,Brasil,Menos de 24 horas depois de a juíza federal Iv...,https://brasil.elpais.com/brasil/2019/03/30/po...,"[justiça, suspende, decisão, proibia, forças, ...","[[-0.014938265, 0.024144838, -0.028129218, 0.0..."
2,Governo Bolsonaro prega “negacionismo históric...,Marcos Napolitano professor da USP diz que o...,Regiane Oliveira,04/04/2019 22:37:48,Brasil,Quando determinou que de 31 de março 1964 u...,https://brasil.elpais.com/brasil/2019/04/05/po...,"[governo, bolsonaro, prega, negacionismo, hist...","[[-0.048315603, -0.0596106, -0.093798175, 0.09..."
3,Quando os pais de Gabo perceberam que tinham u...,Gustavo Tatis percorre o universo de García Má...,Jesús Ruiz Mantilla,07/03/2019 16:38:56,Cultura,Quando era pequeno Luisa e Gabriel se preo...,https://brasil.elpais.com/brasil/2019/03/06/cu...,"[quando, pais, gabo, perceberam, filho, mentir...","[[-0.030334732, -0.0036477367, -0.05803683, -0..."
4,Rádios canadenses banem músicas de Michael Jac...,Quebec Cogeco Media toma a decisão após queixa...,Jaime Porras Ferreyra,07/03/2019 16:12:37,Cultura,Desde a manhã da última segunda-feira e ...,https://brasil.elpais.com/brasil/2019/03/06/cu...,"[rádios, canadenses, banem, músicas, michael, ...","[[-0.04972329, -0.03313186, 0.04639487, 0.0441..."


## Implemente uma função que calcula o WMD (Word Mover's Distance) entre duas notícias usando os embeddings das palavras dos respectivos títulos

In [127]:
def calc_wmd(doc1, doc2):
    words1 = data.embeddings[doc1]
    words2 = data.embeddings[doc2]
  
    wmd = 0
    for w1 in words1:
        min_dist = inf
        for w2 in words2:
            try:
                min_dist = min(min_dist, sqrt(sum((w1 - w2) ** 2)))
            except:
                pass
        wmd += min_dist * 1.0/len(words1)
 
    return wmd
# Noticias nao similares
wmd = calc_wmd(30, 32)
print(data.title[30])
print(data.title[32])
print("WMD: ", wmd)

Matando como em ‘Call of Duty’
A ética da solidariedade feminista
WMD:  1.4371360696829516


In [122]:
# Noticias MUITO similares:

wmd = calc_wmd(84,43)
print(data.title[84])
print(data.title[43])
print("WMD: ", wmd)

Onde e como assistir a Real Madrid x Barcelona pelas semifinais da Copa do Rei
Onde e como assistir a Real Madrid x Barcelona pelo Campeonato Espanhol
WMD:  0.27940795748299774


## Implemente uma função que possa receber qualquer notícia como entrada e retornar as top-3 notícias mais similares (menos distantes) a ela

In [129]:
def top3similar(doc):
    res = []
    for entry in range(len(data.embeddings)):
        if entry != doc:
            wmd = calc_wmd(doc, entry)
            bisect.insort(res, (wmd, entry))
    return res[:3]

print("Search: ", data.title[84])
r = top3similar(84)
print("Results: ")

[print(data.title[i[1]]) for i in r]
None

Search:  Onde e como assistir a Real Madrid x Barcelona pelas semifinais da Copa do Rei
Results: 
Onde e como assistir a Real Madrid x Barcelona pelo Campeonato Espanhol
Com cavadinha de Suárez  Barcelona elimina Real Madrid e vai à final da Copa do Rei
Onde e como assistir a Juventus x Atlético de Madrid pela Champions League
